# Capstone Project - Neighborhood Segmentation & Clustering

In [2]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Scraping Data From The Internet

Install BeautifulSoup 4 and lxml if not already installed

In [3]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


Import BeautifulSoup4 and requests

In [4]:
from bs4 import BeautifulSoup
import requests

Download the info from the website url

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Save the info into a variable named "table"

In [4]:
table = pd.read_html(url)

Inspect the contents

In [25]:
table[0:]

[    Postcode           Borough          Neighbourhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 283      M8Z         Etobicoke              Mimico NW
 284      M8Z         Etobicoke     The Queensway West
 285      M8Z         Etobicoke  Royal York South West
 286      M8Z         Etobicoke         South of Bloor
 287      M9Z      Not assigned           Not assigned
 
 [288 rows x 3 columns],
                                                   0   \
 0                                                NaN   
 1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
 2                                                 NL   
 3                                                  A   
 
                          

Save the contents of the first column only to a dataframe, df and print the first five rows

In [5]:
df = table[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Wrangle the dataframe to its intended form

Only process cells that have an named Borough. Drop all rows where Borough is "Not assigned"

In [6]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)

Print the new dataframe to check

In [7]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [10]:
df.isnull() # Check for nans

,Postcode,Borough,Neighbourhood
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
6,False,False,False
...,...,...,...
282,False,False,False
283,False,False,False
284,False,False,False
285,False,False,False


Combine rows with the same postcodes

In [23]:
dfNew = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)

dfNew = dfNew.sample(frac=1).reset_index()

dfNew['Neighbourhood']= dfNew['Neighbourhood'].str.join(', ')

In [24]:
dfNew.head()

,Postcode,Borough,Neighbourhood
0,M4A,North York,Victoria Village
1,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights"
2,M9A,Etobicoke,Islington Avenue
3,M1G,Scarborough,Woburn
4,M5E,Downtown Toronto,Berczy Park


In [47]:
dfNew.shape

(103, 3)

Replace each instance of "Not assigned" in Neighbourhood with its corresponding Borough.

In [44]:
dfNew['Neighbourhood'].replace(to_replace ="Not assigned", value=dfNew['Borough'], inplace=True)

In [45]:
dfNew.head()

,Postcode,Borough,Neighbourhood
0,M4A,North York,Victoria Village
1,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights"
2,M9A,Etobicoke,Islington Avenue
3,M1G,Scarborough,Woburn
4,M5E,Downtown Toronto,Berczy Park


Check the size of the dataframe

In [46]:
dfNew.shape

(103, 3)